In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import eye
from scipy.sparse.linalg import inv

from ipynb.fs.full.roc_curve import roc_curve

In [2]:
# pd. set_option('display.max_rows', 10) # or 1000.
pd.options.display.max_colwidth = 150

In [3]:
healthy_df = pd.read_excel('features_30_9_healthy.xlsx', header=None)

In [4]:
healthy_df['class'] = 0  # negative
healthy_df.rename(columns={32: 'file_name', 31: 32}, inplace=True)

In [5]:
healthy_df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,32,file_name,class
0,0.09242,0.58207,0.84332,1.85755,0.75574,9.51179,6.68282,3.31333,3.98427,0.32795,...,4539.14893,9787,12757,5068,7689,1721357.125,1312.00500,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
1,0.06342,0.96902,0.74959,1.93489,0.67394,6.65879,6.44094,3.35085,4.40313,0.47218,...,2038.07898,10136,12932,8039,4893,702499.750,838.15259,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
2,0.10062,0.68576,0.79684,1.68772,0.73787,9.50464,6.04032,3.17706,3.93885,0.38197,...,4157.31250,9961,12583,5592,6991,1237559.375,1112.45642,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
3,0.05449,0.74973,0.87516,3.00270,0.71925,7.70397,10.57323,3.66807,4.57908,0.39824,...,3706.43335,8388,12233,4544,7689,2748470.500,1657.85120,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
4,0.07776,0.70084,0.85559,2.42659,0.73281,8.71716,8.64065,3.42859,4.24003,0.38487,...,4775.73877,9088,12583,3845,8738,2867137.750,1693.26245,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,3821.18237,5915,10883,2129,8754,3848079.250,1961.65222,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,4307.60449,5678,10646,1656,8990,3530869.500,1879.06079,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,3502.35352,4495,10646,1420,9226,3042845.750,1744.37549,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,2767.90771,4022,10410,1656,8754,3174905.250,1781.82642,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0


In [6]:
insult_df = pd.read_excel('features_30_9_insult.xlsx', header=None)

In [7]:
insult_df['class'] = 1
insult_df.rename(columns={33: 'file_name'}, inplace=True)

In [8]:
insult_df  # positive

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,file_name,class
0,0.06081,0.75858,0.85508,2.61725,0.71279,9.07447,9.53269,3.54796,4.38729,0.39242,...,7689,10486,4019,6467,1706812.750,1306.45044,0.09679,0.44111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
1,0.05805,0.80593,0.86047,2.88797,0.70327,8.43171,10.18995,3.56138,4.49190,0.41046,...,7340,10486,4019,6467,1868834.875,1367.05334,0.14702,0.57444,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
2,0.05735,0.80487,0.86085,2.89208,0.69992,8.27271,10.48810,3.61427,4.52479,0.40255,...,6990,10311,3845,6466,1798897.125,1341.22974,0.19660,0.60778,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
3,0.07138,0.60490,0.87878,2.49509,0.74577,7.20000,9.16320,3.48943,4.20273,0.32976,...,7340,11884,3845,8039,2424746.750,1557.15979,0.24542,0.55333,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
4,0.06371,0.61074,0.88761,2.71714,0.74178,8.10054,9.98028,3.60307,4.32808,0.32754,...,8214,12058,3845,8213,2890672.500,1700.19775,0.29329,0.48111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,0.06646,0.65228,0.89100,2.99205,0.75269,8.06304,10.62932,3.63314,4.42857,0.38109,...,6861,11120,2129,8991,3640547.750,1908.02197,0.55711,0.62222,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1
5931,0.06672,0.58668,0.89720,2.85336,0.75804,8.28487,10.18638,3.62216,4.34481,0.33563,...,6861,11120,1893,9227,3537324.750,1880.77771,0.45430,0.67111,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1
5932,0.07679,0.60400,0.87018,2.32624,0.75293,8.35847,8.05221,3.45328,4.18973,0.34140,...,6861,11120,1893,9227,2920900.750,1709.06421,0.34919,0.69000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1
5933,0.07262,0.62511,0.87357,2.47208,0.74652,8.93944,8.51236,3.49029,4.25676,0.34790,...,7098,11120,1893,9227,3107574.750,1762.83142,0.24275,0.63889,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1


In [9]:
df = pd.concat([insult_df, healthy_df])

In [10]:
file_names = df['file_name']
unique_file_names = file_names.unique()  # unique file names in df
df.sort_values(by=['file_name', 'class'], ascending=[True, False], inplace=True)

In [11]:
file_names = df['file_name']  # all file names of healthy_df
unique_file_names = file_names.unique()  # unique file names of healthy_df
df['files_indices'] = file_names.factorize()[0]  # index of unique file names
files_indices_list = df['files_indices']
n_files = len(unique_file_names)  # number of unique files

In [12]:
df

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,file_name,class,files_indices
0,0.06081,0.75858,0.85508,2.61725,0.71279,9.07447,9.53269,3.54796,4.38729,0.39242,...,10486,4019,6467,1706812.750,1306.45044,0.09679,0.44111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
1,0.05805,0.80593,0.86047,2.88797,0.70327,8.43171,10.18995,3.56138,4.49190,0.41046,...,10486,4019,6467,1868834.875,1367.05334,0.14702,0.57444,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
2,0.05735,0.80487,0.86085,2.89208,0.69992,8.27271,10.48810,3.61427,4.52479,0.40255,...,10311,3845,6466,1798897.125,1341.22974,0.19660,0.60778,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
3,0.07138,0.60490,0.87878,2.49509,0.74577,7.20000,9.16320,3.48943,4.20273,0.32976,...,11884,3845,8039,2424746.750,1557.15979,0.24542,0.55333,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
4,0.06371,0.61074,0.88761,2.71714,0.74178,8.10054,9.98028,3.60307,4.32808,0.32754,...,12058,3845,8213,2890672.500,1700.19775,0.29329,0.48111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,10883,2129,8754,3848079.250,1961.65222,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,10646,1656,8990,3530869.500,1879.06079,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,10646,1420,9226,3042845.750,1744.37549,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,10410,1656,8754,3174905.250,1781.82642,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45


In [13]:
X0 = df.iloc[:,:31]  # healthy_df feautures

In [14]:
X0

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,0.06081,0.75858,0.85508,2.61725,0.71279,9.07447,9.53269,3.54796,4.38729,0.39242,...,60.42937,0.03730,4.83596,3644.60889,7689,10486,4019,6467,1706812.750,1306.45044
1,0.05805,0.80593,0.86047,2.88797,0.70327,8.43171,10.18995,3.56138,4.49190,0.41046,...,38.84860,0.03628,4.89183,3413.93018,7340,10486,4019,6467,1868834.875,1367.05334
2,0.05735,0.80487,0.86085,2.89208,0.69992,8.27271,10.48810,3.61427,4.52479,0.40255,...,36.57544,0.03675,4.85308,3305.78662,6990,10311,3845,6466,1798897.125,1341.22974
3,0.07138,0.60490,0.87878,2.49509,0.74577,7.20000,9.16320,3.48943,4.20273,0.32976,...,28.97285,0.03326,5.07144,3614.73560,7340,11884,3845,8039,2424746.750,1557.15979
4,0.06371,0.61074,0.88761,2.71714,0.74178,8.10054,9.98028,3.60307,4.32808,0.32754,...,35.43299,0.02961,5.22304,4149.30322,8214,12058,3845,8213,2890672.500,1700.19775
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,14.36549,0.03582,4.90722,3821.18237,5915,10883,2129,8754,3848079.250,1961.65222
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,27.55488,0.03788,4.86279,4307.60449,5678,10646,1656,8990,3530869.500,1879.06079
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,16.21404,0.04572,4.70639,3502.35352,4495,10646,1420,9226,3042845.750,1744.37549
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,5.78366,0.05878,4.47165,2767.90771,4022,10410,1656,8754,3174905.250,1781.82642


In [15]:
feature_32 = df.iloc[:,31]  # вероятности больных по расст от центра
feature_33 = df.iloc[:,32]  # вероятности больных по пересеч с маской

In [16]:
feature_33.where(feature_33<0.95, other=1, inplace=True)
big_prob_feature_33 = feature_33 > 0.05  # there is no feature less than 0.05
feature_33 = feature_33[big_prob_feature_33]
Y = feature_33

In [17]:
mu = X0.mean()
sig = X0.std()
X = (X0 - mu)/sig
X

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-0.658769,-0.388568,0.998650,0.893271,0.168592,1.007406,0.996314,1.102418,0.480440,-0.489884,...,-0.333025,-1.016576,0.676326,-1.419898,-1.606353,-1.792862,-0.994495,-1.588028,-1.001290,-1.372388
1,-0.768904,-0.176755,1.068378,1.325875,-0.045791,0.465373,1.266773,1.169118,0.840055,-0.277057,...,-0.589243,-1.096587,0.822323,-1.452464,-1.638076,-1.792862,-0.994495,-1.588028,-0.995633,-1.349013
2,-0.796837,-0.181497,1.073294,1.332442,-0.121230,0.331290,1.389460,1.431993,0.953120,-0.370375,...,-0.616231,-1.059719,0.721063,-1.467731,-1.669890,-1.803664,-1.017111,-1.588102,-0.998075,-1.358973
3,-0.236982,-1.076033,1.305248,0.698062,0.911277,-0.573315,0.844271,0.811511,-0.154018,-1.229118,...,-0.706493,-1.333484,1.291670,-1.424115,-1.638076,-1.706570,-1.017111,-1.471346,-0.976223,-1.275687
4,-0.543047,-1.049908,1.419478,1.052893,0.821425,0.186101,1.180495,1.376327,0.276895,-1.255308,...,-0.629795,-1.619799,1.687822,-1.348647,-1.558632,-1.695829,-1.017111,-1.458430,-0.959956,-1.220516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,-0.629240,-0.661711,1.457382,1.802215,0.808364,-0.720216,1.916966,1.490393,0.612618,-0.427947,...,-0.879920,-1.132671,0.862539,-1.394970,-1.767604,-1.768357,-1.240160,-1.418275,-0.926528,-1.119671
2104,-0.663956,-0.383961,1.157511,1.286756,0.292898,-0.176193,1.356137,1.373444,0.774842,-0.349730,...,-0.723328,-0.971079,0.746437,-1.326298,-1.789147,-1.782986,-1.301641,-1.400757,-0.937603,-1.151527
2105,0.763015,-0.636213,1.048844,0.694770,0.955640,-1.575607,0.837618,0.075074,-0.506346,-0.231518,...,-0.857973,-0.356089,0.337741,-1.439981,-1.896678,-1.782986,-1.332317,-1.383240,-0.954642,-1.203477
2106,1.961335,-0.825928,1.334743,1.150081,1.144576,-2.288238,1.254391,-0.926722,-1.152835,-0.468531,...,-0.981808,0.668371,-0.275668,-1.543667,-1.939672,-1.797553,-1.301641,-1.418275,-0.950032,-1.189031


In [18]:
n = len(df)

In [19]:
X_train = X[df['files_indices']<32]
X_test = X[df['files_indices']>=32]

In [20]:
y_train = Y[df['files_indices']<32]
y_test = Y[df['files_indices']>=32]

In [21]:
y_train[-60:]

4216    0.65111
4217    0.58444
4218    0.50778
4219    0.48444
4220    0.67333
4221    0.80778
4222    0.76111
4223    0.67667
4224    0.58444
4225    0.33444
4226    0.46889
4227    0.57444
4228    0.52778
1399    1.00000
1400    1.00000
1401    1.00000
1402    1.00000
1403    1.00000
1404    1.00000
1405    1.00000
1406    1.00000
1407    1.00000
1408    1.00000
1409    1.00000
1410    1.00000
1411    1.00000
1412    1.00000
1413    1.00000
1414    1.00000
1415    1.00000
1416    1.00000
1417    1.00000
1418    1.00000
1419    1.00000
1420    1.00000
1421    1.00000
1422    1.00000
1423    1.00000
1424    1.00000
1425    1.00000
1426    1.00000
1427    1.00000
1428    1.00000
1429    1.00000
1430    1.00000
1431    1.00000
1432    1.00000
1433    1.00000
1434    1.00000
1435    1.00000
1436    1.00000
1437    1.00000
1438    1.00000
1439    1.00000
1440    1.00000
1441    1.00000
1442    1.00000
1443    1.00000
1444    1.00000
1445    1.00000
Name: 32, dtype: float64

In [22]:
y_test[:40]

4229    0.49778
4230    1.00000
4231    1.00000
4232    0.85111
4233    0.69333
4234    0.46889
4235    1.00000
4236    1.00000
4237    1.00000
4238    1.00000
4239    0.84667
4240    0.66444
4241    1.00000
4242    1.00000
4243    1.00000
4244    1.00000
4245    1.00000
4246    1.00000
4247    0.94778
4248    0.82000
4249    0.62889
4250    0.94556
4251    1.00000
4252    1.00000
4253    1.00000
4254    1.00000
4255    1.00000
4256    1.00000
4257    1.00000
4258    0.92111
4259    0.76556
4260    0.53222
4261    0.68556
4262    0.89333
4263    1.00000
4264    1.00000
4265    1.00000
4266    1.00000
4267    1.00000
4268    1.00000
Name: 32, dtype: float64

In [23]:
def roc_curve(class_1, class_2, disp_data=1):  # negative, positive
#     rng(1337,'twister')
#     s = rng
#     class_1 = rand(1,3);
#     rng(s)
#     class_2 = 0.5*rand(1,5);
#     sispt = 1;
#     dispp = 1;

#     np.random.seed(1337)
#     class_1 = np.random.rand(1,3)
#     np.random.seed(1337)
#     class_2 = 0.5*np.random.rand(1,3)

    # Calculating the threshold values between the data points
    s_data = np.sort(np.unique(np.concatenate([class_1, class_2], axis=0)))
    s_data = s_data[s_data!=np.array(None)]
    d_data = np.diff(s_data)
    if np.all(d_data==0):
        print('Both class are the same!')
    d_data = np.append(d_data, d_data[-1])
    thres = np.empty(len(s_data)+1)
    thres[0] = s_data[0] - d_data[0]
    thres[1:len(thres)] = s_data + d_data/2

    # Calculating the sensibility and specificity of each threshold
    curve = np.zeros((len(thres),2))
    distance = np.zeros((len(thres),1))
    for id_t in range(len(thres)):
        TP = np.sum(class_2 >= thres[id_t])
        FP = np.sum(class_1 >= thres[id_t])
        FN = np.sum(class_2 < thres[id_t])
        TN = np.sum(class_1 < thres[id_t])

        curve[id_t, 0] = TP/(TP+FN)  # sensitivity - TPR
        curve[id_t, 1] = TN/(TN+FP)  # specificity - FPR

        distance[id_t] = np.sqrt((1-curve[id_t, 0])**2 + (curve[id_t,1]-1)**2)

    # Optimum threshold and parameters
    #opt = np.min(distance)  # 1
    opt = np.argmin(distance)
    TP = np.sum(class_2 >= thres[opt])
    FP = np.sum(class_1 >= thres[opt])
    FN = np.sum(class_2 < thres[opt])
    TN = np.sum(class_1 < thres[opt])

    if disp_data==1:
        print('\n ROC CURVE PARAMETERS:\n')
        print('Threshold:', thres[opt])
        print('Distance:', distance[opt])
        print('Sensitivity:', curve[opt,0])
        print('Specificity:', curve[opt,1])
        print('Accuracy:', (TP+TN)/(TP+TN+FP+FN))
        print('PPV:', TP/(TP+FP))
        print('NPV:', TN/(TN+FN))
        print('FNR:', FN/(FN+TP))
        print('FPR:', FP/(FP+TN))
        print('FDR:', FP/(FP+TP))
        print('FOR:', FN/(FN+TN))
        print('F1 score:', 2*TP/(2*TP+FP+FN))
        print('TP:', TP)
        print('FP:', FP)
        print('FN:', FN)
        print('TN:', TN)

# DBSCAN

In [109]:
from sklearn.cluster import DBSCAN
from scipy.sparse import hstack

from scipy.sparse import hstack

def clus_en_matrix(X, L):  # 10, 100
    n = len(X)
    c_mem = np.zeros((n,L), dtype=int)
    A_m = [None]*L
    print(len(A_m))
    K_l = np.zeros(L, dtype=int)
    D = csr_matrix((n,n), dtype='int')
#     opts = ...
    for l in range(L):
        C = DBSCAN(eps=l, min_samples=3).fit(X)
        c_mem[:,l] = C.labels_
        print(len(c_mem[:,l]))
        
    A_m[l] = csr_matrix((np.ones(n, dtype='int'),
                (np.arange(n),c_mem[:,l])), shape=(n,L))
        
    A = A_m[1]
    for l in range(1,L):
        A = hstack([A, A_m[l]])
    
    A = np.sqrt(1/L)*A
    return A
    

In [110]:
clus_en_matrix(X, 10, 1)

10
8043
8043
8043
8043
8043
8043
8043
8043
8043
8043
0


ValueError: column index exceeds matrix dimensions

In [102]:
A_m = [1]*1
A_m[0]

1

In [69]:
from sklearn.neighbors import KNeighborsClassifier
dsbcan_clf = DBSCAN()
from sklearn.model_selection import GridSearchCV
param_grid = dict(eps=list(range(1, 5)), min_samples=list(range(1, 5)))
  
# defining parameter range
grid = GridSearchCV(dsbcan_clf, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(X)
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )
#DBSCAN(eps=3, min_samples=3).fit(X).labels_

# KNN

In [23]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)

# knn.fit(X_train, y_train_encoded)
# y_pred = knn.predict(X_test)

In [ ]:
# for KNN
from sklearn.neighbors import KNeighborsClassifier
from scipy.sparse import hstack

def clus_en_matrix(X, K):  # X, K - number of kneighbors
    n = len(X)
    c_mem = np.zeros((n,L), dtype=int)
    A_m = [None]*L
    K_l = np.zeros(L, dtype=int)
    D = csr_matrix((n,n), dtype='int')
#     opts = ...
    for l in range(L):
        C = KNeighborsClassifier(n_neighbors=K).fit(X)
        c_mem[:,l] = C.labels_
        
    for l in range(L):
        A_m[l] = csr_matrix((np.ones(n, dtype='int'),
                    (np.arange(n),c_mem[:,l])), shape=(n,K+l))
        
    A = A_m[1]
    for l in range(1,L):
        A = hstack([A, A_m[l]])
    
    A = np.sqrt(1/L)*A
    return A


In [25]:
for n_file in range(n_files):
    inp_files = [i for i in range(n_files) if i!=n_file] + [n_file]
    inp = []
    for j in range(n_files):
        ind_files = [i for i, x in enumerate(indexes) if x==inp_files[j]]  # all indexes with number 1,2,3,...,46,1
        inp.extend(ind_files)  # concatenate indexes in loop
    new_indexes = indexes[inp]  # новый порядок объектов
    
    # какие объекы в последних контрольных снимках
    tes_test = np.isin(indexes, inp_files[len(inp_files)-n_test_files:len(inp_files)])  # inp_files[46:46]
    n_test_0 = sum(tes_test)  # количество объектов в контрольной выборке = 277
        
    # какие объекы в неточных снимках
    tes_uncertain = np.isin(indexes, inp_files[n_certain_files:n_certain_files+n_uncertain_files])  # 1,2,...,45
    n_uncertain = sum(tes_uncertain)  # количество объектов в неточной выборке = 7766
        
    # какие объекы в точных снимках
    tes_certain = np.isin(indexes, inp_files[:n_certain_files])  # empty
    n_certain = sum(tes_certain)  # количество объектов в точной выборке = 0
        
    X = X0.iloc[inp]
    Y_pos = Y_pos_0.iloc[inp]
    Y1M = Y1M0.iloc[inp]
    
    ind_insult = Y1M==1  # уверенно классиф как больные
    ind_healthy = Y1M==0  # уверенно классиф как здоровые
    ind_certain = np.logical_or(ind_insult,ind_healthy)  # 4958 уверенно классифицированных данных
    
    Y10Tr = np.array([[*ind_insult[:n_certain], *Y_pos[n_certain:n_certain+n_uncertain], *pd.Series(np.zeros((n_test_0,), dtype=int))],
                      [*ind_healthy[:n_certain], *(1-Y_pos[n_certain:n_certain+n_uncertain]), *pd.Series(np.zeros((n_test_0,), dtype=int))]]).T
    
    Y1MC = Y1M[n-n_test_0:len(Y1M)]  # 8043-277 : 8043 (last)
    Y_true_test = 1+(Y1MC[ind_certain[n-n_test_0:len(ind_certain)]]==0)
    
#     Y10Tr2 = Y10Tr
#     ind_uncertain = np.logical_not(ind_certain)
#     Y10Tr2[ind_uncertain][n_certain:n-n_test_0, :] = 0
    
    #print(Y10Tr2)
    
    sigma = 1.25
    bet = 0.001
    mmax = 1000
    G = bet * eye(n, dtype='int') + csr_matrix(([*pd.to_numeric(ind_certain[:n_certain], errors='coerce'),
                 *np.ones(n_uncertain, dtype='int'),
                 *np.zeros(n_test_0, dtype='int')],
                (np.arange(n),np.arange(n))), shape=(n,n))
    G2 = bet * eye(n, dtype='int') + csr_matrix(([*pd.to_numeric(ind_certain[:n-n_test_0], errors='coerce'),
                  *np.zeros(n_test_0, dtype='int')],
                (np.arange(n),np.arange(n))), shape=(n,n))
    
    A = clus_en_matrix(X,10,100)
    m = A.shape[1]  # 1046 in order of 1055
    u = A.sum(axis = 0)  # by columns
    dd = A * u.conj().transpose()  # 8043x1 array
    dd[dd<0] = 0
    D = csr_matrix(([item for sublist in dd.tolist() for item in sublist], 
                    (np.arange(n), np.arange(n))), shape=(n,n), dtype='int')
    D = csr_matrix((1/np.sqrt(D.diagonal()),
                    (np.arange(n), np.arange(n))), shape=(n,n))
    U = D*A
    Gm = csr_matrix((1/G.diagonal(),
                    (np.arange(n), np.arange(n))), shape=(n,n))
    A1 = bet * Gm * U
    A2 = eye(m, dtype='int') - bet*(U.conj().transpose() * Gm)*U

    F = np.empty(shape=[n,K]) 
    for k in range(K):
        A3 = inv(A2) * (U.conj().transpose() * Gm * Y10Tr[:,k])
        F[:,k] = Gm * Y10Tr[:,k] + A1 * A3
        
    F = F/F.sum(axis = 1).reshape((n,1))  # нормировка
    Ff = F[n-n_test_0:,]
    Ft = Ff[ind_certain[n-n_test_0:],]

    class11 = Ft[Y_true_test==2, 0]  # negative
    class21 = Ft[Y_true_test==1, 0]  # positive
    roc_curve(class11, class21, 1)
    print(n_file)


NameError: name 'indexes' is not defined

In [26]:
# ValueError: Unknown label type: 'continuous'
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_train_encoded

array([ 89, 204, 234, ..., 536, 536, 536], dtype=int64)

In [32]:
y_train_encoded[-60:]

array([272, 213, 146, 126, 292, 409, 370, 295, 213,  28, 112, 204, 164,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536], dtype=int64)

In [35]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 30))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train_encoded)

Fitting 10 folds for each of 29 candidates, totalling 290 fits


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


In [37]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

{'n_neighbors': 29}
Accuracy for our training dataset with tuning is : 61.78%


In [39]:
knn = KNeighborsClassifier(n_neighbors=29)
knn.fit(X_train, y_train_encoded)
y_pred=knn.predict(X_test) 
y_pred

array([536, 536, 536, ..., 536, 536, 536], dtype=int64)

In [40]:
lab_enc.fit_transform(y_test)

array([ 79, 433, 433, ..., 433, 433, 433], dtype=int64)

In [41]:
from sklearn.metrics import accuracy_score, plot_confusion_matrix
test_accuracy=accuracy_score(lab_enc.fit_transform(y_test),y_pred)*100
print("Accuracy for our testing dataset with tuning is : {:.2f}%".format(test_accuracy) )

Accuracy for our testing dataset with tuning is : 0.00%


In [43]:
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

print(classification_report(lab_enc.fit_transform(y_test), y_pred))
#Checking performance our model with ROC Score.
roc_auc_score(lab_enc.fit_transform(y_test), y_pred)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00       1.0
           1       0.00      0.00      0.00       1.0
           2       0.00      0.00      0.00       1.0
           3       0.00      0.00      0.00       1.0
           4       0.00      0.00      0.00       1.0
           5       0.00      0.00      0.00       2.0
           6       0.00      0.00      0.00       2.0
           7       0.00      0.00      0.00       2.0
           8       0.00      0.00      0.00       1.0
           9       0.00      0.00      0.00       2.0
          10       0.00      0.00      0.00       1.0
          11       0.00      0.00      0.00       1.0
          12       0.00      0.00      0.00       1.0
          13       0.00      0.00      0.00       1.0
          14       0.00      0.00      0.00       1.0
          15       0.00      0.00      0.00       1.0
          16       0.00      0.00      0.00       1.0
          17       0.00    

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\

ValueError: multi_class must be in ('ovo', 'ovr')

In [86]:
t = [i for i, x in enumerate(df[df['files_indices']>=32]['class']==0) if x]
y_pred[X_test.index.isin(t)]

array([536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 350, 536, 536, 180, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536,
       536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 536, 53

In [95]:
y_pred[~X_test.index.isin(t)]

array([536, 536, 536, ..., 536, 536, 536], dtype=int64)

In [105]:
roc_curve(y_pred[df['class']==0], y_pred[df['class']==1])

IndexError: boolean index did not match indexed array along dimension 0; dimension is 2368 but corresponding boolean dimension is 8043

In [110]:
y_pred[df['class']==0]

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2103    1.0
2104    1.0
2105    1.0
2106    1.0
2107    1.0
Name: 32, Length: 2108, dtype: float64

In [124]:
df.loc[(df['files_indices']>=32) & (df['class']==0), :]

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,file_name,class,files_indices
1446,0.05333,1.10926,0.78470,2.57606,0.65157,9.14565,8.51365,3.53951,4.70391,0.52400,...,56912,27594,29318,31191230.00,5584.91064,NaN,1.0,VVG15051946__20180724__11069__CT__HEAD__0004__brain 1 mm_1.2.840.113704.1.111.5240.1532410506.8795_219,0,32
1447,0.08045,0.96369,0.67386,1.47743,0.68072,9.02086,4.88575,3.17193,4.18123,0.48431,...,53463,25007,28456,16596833.00,4073.92114,NaN,1.0,VVG15051946__20180724__11069__CT__HEAD__0004__brain 1 mm_1.2.840.113704.1.111.5240.1532410506.8795_219,0,32
1448,0.08401,0.76972,0.76791,1.65825,0.71116,8.17358,5.58876,3.23694,4.11483,0.39801,...,58637,23282,35355,33329096.00,5773.13574,NaN,1.0,VVG15051946__20180724__11069__CT__HEAD__0004__brain 1 mm_1.2.840.113704.1.111.5240.1532410506.8795_219,0,32
1449,0.04957,1.43643,0.64837,2.04250,0.61214,7.60650,6.70359,3.39093,4.66595,0.65295,...,51738,29318,22420,14905928.00,3860.81958,NaN,1.0,VVG15051946__20180724__11069__CT__HEAD__0004__brain 1 mm_1.2.840.113704.1.111.5240.1532410506.8795_219,0,32
1450,0.07559,0.75993,0.84205,2.40555,0.71816,9.53440,8.42093,3.41253,4.28450,0.40425,...,56912,16384,40528,58949284.00,7677.84375,NaN,1.0,VVG15051946__20180724__11069__CT__HEAD__0004__brain 1 mm_1.2.840.113704.1.111.5240.1532410506.8795_219,0,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,10883,2129,8754,3848079.25,1961.65222,NaN,1.0,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,10646,1656,8990,3530869.50,1879.06079,NaN,1.0,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,10646,1420,9226,3042845.75,1744.37549,NaN,1.0,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,10410,1656,8754,3174905.25,1781.82642,NaN,1.0,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45


# Spectral

In [31]:
from scipy.sparse import hstack
from sklearn.cluster import SpectralClustering
sc = SpectralClustering(n_clusters=2)

def clus_en_matrix(X, L, K):
    n = len(X)
    c_mem = np.zeros((n,L), dtype=int)
    A_m = [None]*L
    K_l = np.zeros(L, dtype=int)
    D = csr_matrix((n,n), dtype='int')
#     opts = ...
    for l in range(L):
        C = sc.fit(X)
        c_mem[:,l] = C.labels_
        
    for l in range(L):
        A_m[l] = csr_matrix((np.ones(n, dtype='int'),
                    (np.arange(n),c_mem[:,l])), shape=(n,K+l))
        
    A = A_m[1]
    for l in range(1,L):
        A = hstack([A, A_m[l]])
    
    A = np.sqrt(1/L)*A
    return A


In [ ]:
y_pred = sc.predict(X_test)

In [ ]:
from sklearn import metrics
# Model Accuracy, how often is the classifier correct?
print("Accuracy:",metrics.accuracy_score(lab_enc.fit_transform(y_test), y_pred))

In [ ]:
from sklearn.neighbors import SpectralClustering
knn = KNeighborsClassifier()
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 100))
param_grid = dict(n_neighbors=k_range)
  
# defining parameter range
grid = GridSearchCV(knn, param_grid, cv=10, scoring='accuracy', return_train_score=False, verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train_encoded)

n_clusters=2,
        assign_labels='discretize',
        random_state=0

In [ ]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

# OPTICS

In [22]:
# ValueError: Unknown label type: 'continuous'
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_train_encoded

array([ 89, 204, 234, ..., 536, 536, 536], dtype=int64)

In [26]:
from sklearn.cluster import OPTICS
opt = OPTICS()
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 30))
param_grid = dict(min_samples=k_range)
  
# defining parameter range
grid = GridSearchCV(opt, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train)

Fitting 10 folds for each of 29 candidates, totalling 290 fits


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 672, in _score
    scores = scorer(estimator, X_test)
TypeError: __call__() missing 1 required positional argument: 'y_true'

  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:683: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 672, in _score
    scores = scorer(estimator, X_test)
TypeError: __call__() missing 1 required positional arg

KeyboardInterrupt: 

In [ ]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

# MLP

In [29]:
# ValueError: Unknown label type: 'continuous'
from sklearn import preprocessing
from sklearn import utils

lab_enc = preprocessing.LabelEncoder()
y_train_encoded = lab_enc.fit_transform(y_train)
y_train_encoded

array([ 89, 204, 234, ..., 536, 536, 536], dtype=int64)

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
from sklearn.model_selection import GridSearchCV
param_grid = [
        {
            'activation' : ['identity', 'logistic', 'tanh', 'relu'],
            'solver' : ['lbfgs', 'sgd', 'adam'],
            'hidden_layer_sizes': [
             (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
             ]
        }
       ]

clf = GridSearchCV(MLPClassifier(), param_grid, cv=3,
                           scoring='accuracy')
clf.fit(X_train, y_train_encoded)


print("Best parameters set found on development set:")
print(clf.best_params_)

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:500: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier()
from sklearn.model_selection import GridSearchCV
k_range = list(range(1, 100))
param_grid = dict(max_iter=k_range)
  
# defining parameter range
grid = GridSearchCV(mlp, param_grid, cv=10, scoring='accuracy', return_train_score=False,verbose=1)
  
# fitting the model for grid search
grid_search=grid.fit(X_train, y_train_encoded)

Fitting 10 folds for each of 99 candidates, totalling 990 fits


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (4) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:619: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (7) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (7) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (8) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\si

In [ ]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

# Logistic regression

In [31]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid={"C":np.logspace(-4, 4, 50), "penalty":["l1","l2"], "solver":['liblinear', 'saga']}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X_train,y_train_encoded)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(("The least populated class in y has only %d"


KeyboardInterrupt: 

# Decision tree

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

pipe = Pipeline([('classifier' , RandomForestClassifier())])

# Create param grid.
param_grid = [
    {'classifier' : [LogisticRegression()],
     'classifier__penalty' : ['l1', 'l2'],
    'classifier__C' : np.logspace(-4, 4, 20),
    'classifier__solver' : ['liblinear']},
    {'classifier' : [RandomForestClassifier()],
    'classifier__n_estimators' : list(range(10,101,10)),
    'classifier__max_features' : list(range(6,32,5))}
]

# Create grid search object
clf = GridSearchCV(pipe, param_grid = param_grid, cv = 5, verbose=True, n_jobs=-1)

# Fit on data
best_clf = clf.fit(X_train, y_train_encoded)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


In [ ]:
print(grid_search.best_params_)
accuracy = grid_search.best_score_ *100
print("Accuracy for our training dataset with tuning is : {:.2f}%".format(accuracy) )

In [43]:
from sklearn import tree
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
#For work encode categorical atrubuts
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
#For do a best a work flow
from sklearn.pipeline import Pipeline
#Missing values
from sklearn.impute import SimpleImputer

from sklearn.model_selection import GridSearchCV

param_grid = {'max_features': ['auto', 'sqrt', 'log2'],
              'ccp_alpha': [0.1, .01, .001],
              'max_depth' : [5, 6, 7, 8, 9],
              'criterion' :['gini', 'entropy']
             }
tree_clas = DecisionTreeClassifier(random_state=1024)
grid_search = GridSearchCV(estimator=tree_clas, param_grid=param_grid, cv=5, verbose=True)
grid_search.fit(X_train, y_train_encoded)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=1024),
             param_grid={'ccp_alpha': [0.1, 0.01, 0.001],
                         'criterion': ['gini', 'entropy'],
                         'max_depth': [5, 6, 7, 8, 9],
                         'max_features': ['auto', 'sqrt', 'log2']},
             verbose=True)

In [44]:
final_model = grid_search.best_estimator_
final_model

DecisionTreeClassifier(ccp_alpha=0.1, max_depth=5, max_features='auto',
                       random_state=1024)

In [47]:
tree_clas = DecisionTreeClassifier(ccp_alpha=0.1, class_weight=None, criterion='entropy',
                       max_depth=5, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       random_state=1024, splitter='best')
tree_clas.fit(X_train, y_train_encoded)
y_predict = tree_clas.predict(X_test)

In [50]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [53]:
confusion_matrix(lab_enc.fit_transform(y_test), y_predict)

array([[   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       [   0,    0,    0, ...,    0,    0,    1],
       ...,
       [   0,    0,    0, ...,    0,    0,    5],
       [   0,    0,    0, ...,    0,    0, 1392],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int64)

In [55]:
accuracy_score(y_test, y_predict)

ValueError: Classification metrics can't handle a mix of continuous and binary targets

In [48]:
print("accuracy :",final_model.best_score_)

AttributeError: 'DecisionTreeClassifier' object has no attribute 'best_score_'